In [ ]:
import pandas as pd
import csv

df_institutions_0 = pd.read_csv("../data/analysis/filtered_types/institutions_0.csv")

df_institutions_1 = pd.read_csv("../data/analysis/filtered_types/institutions_1.csv")


df_institutions_0_inproc = pd.read_csv(
    "../data/analysis/filtered_types/institutions_0_inproc.csv"
)

df_institutions_1_inproc = pd.read_csv(
    "../data/analysis/filtered_types/institutions_1_inproc.csv"
)

df_journals = pd.concat([df_institutions_0, df_institutions_1])
df_conferences = pd.concat(
    [
        df_institutions_0_inproc,
        df_institutions_1_inproc,
    ]
)


df_institutions_0_inproc_count = pd.read_csv(
    "../data/analysis/institutions_0_inproc_count_authors.csv"
)

df_institutions_1_inproc_count = pd.read_csv(
    "../data/analysis/institutions_1_inproc_count_authors.csv"
)

grouped1 = (
    pd.concat([df_institutions_0_inproc, df_institutions_1_inproc])
    .groupby(["display_name"])
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)

grouped2 = (
    pd.concat([df_institutions_0_inproc_count, df_institutions_1_inproc_count])
    .groupby(["display_name"])
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)

# Displaying the table
print(grouped1)
print(grouped2)

In [1]:
def calculate_sum_score(df, file_to_save):
    # Grouping by 'display_name', summing 'score'
    sum_scores = (
        df.groupby(["display_name", "country_code"])["score"]
        .sum()
        .reset_index(name="sum_score")
    )
    sum_scores["sum_score"] = sum_scores["sum_score"].round(3)
    # Grouping by 'display_name' and collecting 'Country' values into lists
    # grouped_countries = df.groupby(["display_name", "country_code"])

    # Merging the two results based on 'display_name'
    result = sum_scores.sort_values(by="sum_score", ascending=False)

    result = result[result["display_name"] != "Software"]

    result["rank"] = result.reset_index().index + 1
    result.to_csv(file_to_save)

In [3]:
# Scored dataset
import pandas as pd

df_institutions_scored_0 = pd.read_csv(
    "../data/analysis/filtered_types/institutions/institutions_0.csv"
)
df_institutions_scored_1 = pd.read_csv(
    "../data/analysis/filtered_types/institutions/institutions_1.csv"
)

df_institutions_scored_0_inproc = pd.read_csv(
    "../data/analysis/filtered_types/institutions/institutions_0_inproc.csv"
)
df_institutions_scored_1_inproc = pd.read_csv(
    "../data/analysis/filtered_types/institutions/institutions_1_inproc.csv"
)

df_journals_scored = pd.concat([df_institutions_scored_0, df_institutions_scored_1])
df_conferences_scored = pd.concat(
    [
        df_institutions_scored_0_inproc,
        df_institutions_scored_1_inproc,
    ]
)


def process_institution_display_name(name):
    if "Microsoft" in name:
        return "Microsoft"
    if "Meta" in name:
        return "Meta"
    if "IBM Research" in name:
        return "IBM Research"
    if "Inria" in name:
        return "Inria"
    name.replace('"', "")
    name_without_country = name.split(" (")[0]
    name_without_country_splitted = name_without_country.split(",")
    result = (
        name_without_country_splitted[0]
        if len(name_without_country_splitted) <= 2
        else name_without_country
    )

    return result


df_journals_scored["display_name"] = df_journals_scored["display_name"].apply(
    process_institution_display_name
)
df_conferences_scored["display_name"] = df_conferences_scored["display_name"].apply(
    process_institution_display_name
)

calculate_sum_score(
    df_journals_scored,
    "../data/analysis/filtered_types/tables/institutions_by_score/journals_only.csv",
)
calculate_sum_score(
    pd.concat([df_journals_scored, df_conferences_scored]),
    "../data/analysis/filtered_types/tables/institutions_by_score/all.csv",
)